In [1]:
import logging
import os
import random
import numpy as np
import pandas as pd

from src.configuration_space.LKH import CONFIGURATION_SPACE
from src.constant import DATA_DIR, MAX_WORKERS, TEMP_DIR
from src.experiment.GlobalExperiment import GlobalExperiment
from src.experiment.ParhydraExperiment import ParhydraExperiment
from src.experiment.CepsMy1Experiment import CepsMy1Experiment
from src.experiment.PcitExperiment import PcitExperiment, _Clustering
from src.instance.TSP_Instance import TSP_Instance, TSP_InstanceSet
from src.portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
from src.experiment.CepsExperiment import CepsExperiment
pd.options.display.max_columns = 999

In [2]:
train_instances, test_instances = TSP_InstanceSet.train_test_from_index_file(
    filepath=DATA_DIR / "TSP" / "CEPS_benchmark" / "index.json",
    train_size=5,
    seed=0,
)

In [3]:
solver_class = TSP_LKH_Solver
instance_class= TSP_Instance
t_c = 300
K = 2
n = 2
t_ini = train_instances.size * 10 * 10
t_i = 10
max_iter = 1

experiment = CepsMy1Experiment(
    t_c=t_c,
    t_ini=t_ini,
    t_i=t_i,
    K=K,
    n=n,
    max_iter=max_iter,
    solver_class=solver_class,
    instance_class=instance_class,
)

best_portfolio = experiment.construct_portfolio(train_instances)

[2024-12-14 23:10:57] INFO      [CEPS_MY_1] Start!
[2024-12-14 23:10:58] INFO      Initialization
[2024-12-14 23:10:58] INFO      Solver 1/2
[2024-12-14 23:10:58] INFO      Random config 1/10
[2024-12-14 23:10:58] DEBUG     =================================  Portfolio  ==================================
[2024-12-14 23:10:58] DEBUG     2220588739511820280 --> ASCENT_CANDIDATES=52;BACKBONE_TRIALS=1;BACKTRACKING=YES;CANDIDATE_SET_TYPE=ALPHA;EXTRA_CANDIDATES=7;EXTRA_CANDIDATE_SET_TYPE=QUADRANT;GAIN23=NO;GAIN_CRITERION=YES;INITIAL_STEP_SIZE=1;INITIAL_TOUR_ALGORITHM=QUICK-BORUVKA;INITIAL_TOUR_FRACTION=0.5820197920751;KICKS=3;KICK_TYPE=0;MAX_BREADTH=954863081;MAX_CANDIDATES=10;MOVE_TYPE=3;PATCHING_A=3;PATCHING_C=1;POPULATION_SIZE=8;RESTRICTED_SEARCH=YES;SUBGRADIENT=YES;SUBSEQUENT_MOVE_TYPE=6;SUBSEQUENT_PATCHING=NO
[2024-12-14 23:10:58] DEBUG     ================================================================================
[2024-12-14 23:10:58] DEBUG     executor start
[2024-12-14 23:11:06]

In [ ]:
# t_c = 60
# K = 2
# n = 2
# solver_class = TSP_LKH_Solver
# instance_class= TSP_Instance

# experiment = GlobalExperiment(
#     t_c=t_c,
#     K=K,
#     n=n,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
# t_c = 50
# K = 2
# n = 2
# max_iter = 2
# solver_class = TSP_LKH_Solver
# instance_class = TSP_Instance

# experiment = PcitExperiment(
#     t_c=t_c,
#     K=K,
#     n=n,
#     max_iter=max_iter,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
# t_c = 50
# K = 2
# n = 2
# max_iter = 2
# solver_class = TSP_LKH_Solver
# instance_class = TSP_Instance

# experiment = PcitExperiment(
#     t_c=t_c,
#     K=K,
#     n=n,
#     max_iter=max_iter,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
# solver_class = TSP_LKH_Solver
# instance_class= TSP_Instance
# t_c = 200
# K = 2
# n = 2
# t_ini = train_instances.size * 10 * 2
# t_i = 10
# max_iter = 2

# experiment = CepsExperiment(
#     t_c=t_c,
#     t_ini=t_ini,
#     t_i=t_i,
#     K=K,
#     n=n,
#     max_iter=max_iter,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
# solver_class = TSP_LKH_Solver
# instance_class= TSP_Instance
# t_c = 200
# K = 2
# n = 2
# t_ini = train_instances.size * 10 * 2
# t_i = 10
# max_iter = 1

# experiment = CepsMy1Experiment(
#     t_c=t_c,
#     t_ini=t_ini,
#     t_i=t_i,
#     K=K,
#     n=n,
#     max_iter=max_iter,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [22]:
# remaining_time = np.ones(shape=(K, )) * np.inf
# best_portfolio.evaluate(test_instances, remaining_time, "test")

In [4]:
from src.database import db_connect
conn = db_connect()

In [84]:
import sqlite3
conn = sqlite3.connect("_archive/7/CEPS2.db")
# conn = sqlite3.connect("database/2024_12_10_22_00_10.db")

In [ ]:
df_solvers = pd.read_sql_query("SELECT * FROM solvers", conn)
df_solvers

In [ ]:
df_instances = pd.read_sql_query("SELECT * FROM instances", conn)
df_instances

In [ ]:
df = pd.read_sql_query("SELECT * FROM results", conn)
df

In [ ]:
df["comment"].value_counts()

In [ ]:
df.loc[df["comment"].str.startswith("phase")]

In [ ]:
df["comment"].value_counts()

In [ ]:
_ = df.loc[df["comment"].str.contains("test")].pivot_table(index=["instance_id", "solver_id"], columns="comment", values="time").sort_values(["instance_id", "solver_id"])
_["range"] = _.max(axis=1) - _.min(axis=1)
_.sort_values(by="range")

In [ ]:
_ = df.loc[df["comment"].str.startswith("test")].pivot_table(index="instance_id", columns="solver_id", values="cost", aggfunc=lambda x: sorted(x)[1])
_["best"] = np.argmin(_.values, axis=1)
_["min"] = _.iloc[:, :-1].min(axis=1)
_

In [ ]:
_["min"].describe()

In [ ]:
(_["min"] > 90).value_counts()

In [ ]:
_.loc[lambda x: x["min"] < 90, "best"].value_counts()

In [ ]:
_.sort_values(by="min")

In [ ]:
# config = df_solvers.loc[df_solvers["id"] == "OJQGNYe__-omEPLuUJkSzOuFdCnYPLzZd99VQqxW2gM="].drop(columns=["id"]).iloc[0].to_dict()
# config = Configuration(configuration_space=CONFIGURATION_SPACE, values=config)
# solver = TSP_LKH_Solver(config=config)

# instance = TSP_Instance(filepath="data/TSP/CEPS_benchmark/cluster_netgen/18.tsp", optimum=10555222.0)

# solver.solve(instance)

In [ ]:
# import json
# with open(DATA_DIR / "TSP" / "index.json") as f:
#     index = json.load(f)

# new_index = {}

# for k, v in index.items():
#     path = k.split("/")
#     file = path[-1]
#     file_no_ext = file.split(".")[0]
#     if "_" in file_no_ext:
#         a, b = file_no_ext.split("_")
#         n = str((int(a) - 4) * 10 + int(b)).zfill(2)
#         new_file = f"{n}.tsp"
#     else:
#         file_no_ext = str(int(file_no_ext)).zfill(2)
#         new_file = f"{file_no_ext}.tsp"
#     new_path = path[:-1] + [new_file]
#     new_k = "/".join(new_path)
#     new_index[new_k] = float(v)

#     path1 = DATA_DIR / k;
#     path2 = DATA_DIR / new_k;
#     path1.rename(path2)

# new_index = {k: new_index[k] for k in sorted(new_index.keys())}

# with open(DATA_DIR / "TSP" / "index.json", "w") as f:
#     json.dump(new_index, f, indent=4)

# new_index